# Strands를 활용한 데이터베이스 전문 에이전트
이 노트북에서는 Strands Agents를 사용하여 사내 데이터베이스 (PostgreSQL)와 연동하는 방법을 실습합니다.

## 학습 목표
- 사내 데이터베이스를 Agent의 도구로 활용하는 방법 이해
- 관계형 데이터베이스의 쿼리를 몰라도 자연어로 쿼리를 실행하고 결과를 획득하는 과정 실습

## 실습 데이터셋 소개

의료 연구자들은 복잡한 데이터베이스 구조와 수많은 컬럼명을 모두 기억하기 어렵습니다.

이 실습에서는 다음과 같은 실제 의료 데이터를 사용하여, LLM의 도움으로 자연어만으로도 복잡한 데이터 분석이 가능함을 보여드립니다.

### chemotherapy_survival 테이블 (화학요법 생존 데이터)
화학요법 후 환자 생존 데이터를 저장하는 테이블입니다.

<img src="https://raw.githubusercontent.com/jikang-jeong/aws-q-mem-mcp-server/refs/heads/main/rdb_sample1.png" width="800" alt="chemotherapy_survival 데이터 예시">

#### 테이블 구조
| 컬럼명 | 데이터 타입 | 설명 |
|--------|-------------|------|
| LRIG1 | FLOAT | LRIG1 유전자 발현량 (종양억제 유전자) |
| Survival_Status | BOOLEAN | 생존 상태 (true=생존, false=사망) |
| Survival_Duration | FLOAT | 생존 기간 (일 단위) |
| ExpressionGroup | FLOAT | 발현 그룹 (0 또는 1로 분류) |

#### 주요 특징
- **목적**: 화학요법 후 환자 생존 분석
- **핵심 분석**: LRIG1 유전자 발현량과 생존율의 상관관계
- **데이터 규모**: 소규모 생존 분석 전용 테이블
---
### clinical_genomic 테이블 (임상-유전체 통합 데이터)
폐암 환자의 임상 정보와 유전체 데이터를 통합한 대규모 테이블입니다.

<img src="https://raw.githubusercontent.com/jikang-jeong/aws-q-mem-mcp-server/refs/heads/main/rdb_sample2.png" width="800" alt="clinical_genomic 데이터 예시">

#### 테이블 구조
| 컬럼명 | 데이터 타입 | 설명 |
|--------|-------------|------|
| Case_ID | VARCHAR(50) | 환자 케이스 식별번호 |
| **유전자 발현 데이터 (21개)** | | |
| LRIG1 | FLOAT | LRIG1 유전자 발현량 (종양억제 유전자) |
| HPGD | FLOAT | 프로스타글란딘 대사 관련 유전자 |
| GDF15 | FLOAT | 성장분화인자 (염증/스트레스 반응) |
| CDH2 | FLOAT | 세포접착분자 (전이 관련) |
| POSTN, VCAN, PDGFRA, VCAM1 | FLOAT | 종양미세환경 관련 유전자들 |
| CD44, CD48, CD4, CD37 | FLOAT | 면역세포 표면 마커 |
| VIM | FLOAT | 비멘틴 (상피간엽전이 마커) |
| COL4A1, COL5A1, COL5A2 | FLOAT | 콜라겐 (세포외기질 구조) |
| ... (기타 유전자들) | FLOAT | LYL1, SPI1, LMO2, EGR2, BGN 등 |
| **환자 기본 정보** | | |
| Patient_affiliation | VARCHAR(50) | 환자 소속 병원/기관 |
| Age_at_Histological_Diagnosis | INT | 조직학적 진단 당시 나이 |
| Weight_lbs | FLOAT | 체중 (파운드) |
| Gender | VARCHAR(10) | 성별 |
| Ethnicity | VARCHAR(50) | 인종/민족 |
| Smoking_status | VARCHAR(50) | 흡연 상태 |
| Pack_Years | INT | 갑년 (하루 담배 갑수 × 흡연 년수) |
| **종양 위치 정보 (폐엽별)** | | |
| Tumor_Location_RUL | VARCHAR(20) | 우상엽 종양 위치 |
| Tumor_Location_RML | VARCHAR(20) | 우중엽 종양 위치 |
| Tumor_Location_RLL | VARCHAR(20) | 우하엽 종양 위치 |
| Tumor_Location_LUL | VARCHAR(20) | 좌상엽 종양 위치 |
| Tumor_Location_LLL | VARCHAR(20) | 좌하엽 종양 위치 |
| Tumor_Location_L_Lingula | VARCHAR(20) | 좌설상구 종양 위치 |
| **병리학적 정보** | | |
| Histology | VARCHAR(70) | 조직학적 암 유형 (선암, 편평세포암 등) |
| Pathological_T_stage | VARCHAR(20) | 원발종양 병기 (T1-T4) |
| Pathological_N_stage | VARCHAR(20) | 림프절 전이 병기 (N0-N3) |
| Pathological_M_stage | VARCHAR(20) | 원격전이 병기 (M0-M1) |
| Histopathological_Grade | VARCHAR(70) | 조직학적 분화도 |
| Lymphovascular_invasion | VARCHAR(60) | 림프관/혈관 침습 여부 |
| Pleural_invasion | VARCHAR(50) | 흉막 침습 여부 |
| **분자 마커 (표적치료 관련)** | | |
| EGFR_mutation_status | VARCHAR(50) | EGFR 유전자 돌연변이 상태 |
| KRAS_mutation_status | VARCHAR(50) | KRAS 유전자 돌연변이 상태 |
| ALK_translocation_status | VARCHAR(50) | ALK 유전자 전좌 여부 |
| **치료 및 예후 정보** | | |
| Adjuvant_Treatment | VARCHAR(20) | 보조 치료 여부 |
| Chemotherapy | VARCHAR(20) | 화학요법 시행 여부 |
| Radiation | VARCHAR(20) | 방사선 치료 여부 |
| Recurrence | VARCHAR(20) | 재발 여부 |
| Recurrence_Location | VARCHAR(50) | 재발 부위 |
| Survival_Status | BOOLEAN | 생존 상태 |
| Time_to_Death | FLOAT | 사망까지의 시간 |
| Days_between_CT_and_surgery | INT | CT와 수술 간 간격 |
| Survival_Duration | FLOAT | 전체 생존 기간 |

#### 주요 특징
- **목적**: 폐암 환자의 종합적 임상-유전체 분석
- **데이터 규모**: 대규모 다차원 임상 연구용 (50+ 컬럼)
- **활용**: 개인맞춤 치료, 예후 예측, 바이오마커 발굴

### 연구자의 도전과 Text2SQL의 해결책

**🤔 연구자의 고민**: 
- "EGFR 돌연변이 환자의 생존율이 궁금한데, 어떤 컬럼을 조회해야 하지?"
- "50세 이상 흡연자 중 선암 환자는 몇 명일까? SQL을 어떻게 작성하지?"
- "LRIG1 유전자 발현량과 생존 기간의 상관관계를 보려면...?"

** Text2SQL로 해결**: 복잡한 50+ 컬럼 구조도 자연어 한 문장으로 쿼리 가능

## Text2SQL 개념 이해

DB Agent의 핵심 기능인 Text2SQL(자연어를 SQL로 변환)에 대해 알아보겠습니다.

### Text2SQL 처리 과정

```
자연어 질문 → 스키마 분석 → SQL 생성 → 쿼리 실행 → 결과 해석
```

### 실제 변환 예시

**예시 1: 기본 통계 쿼리**
```
자연어: "선암 환자의 평균 연령은?"
↓ 스키마 분석
- 테이블: clinical_genomic
- 컬럼: Histology (조직학), Age_at_Histological_Diagnosis (진단시 나이)
- 조건: Histology에서 'Adenocarcinoma' 포함
↓ SQL 생성
SELECT AVG(Age_at_Histological_Diagnosis) 
FROM clinical_genomic 
WHERE Histology LIKE '%Adenocarcinoma%'
```

**예시 2: 복합 조건 쿼리**
```
자연어: "50세 이상 흡연자 중 EGFR 돌연변이가 있는 환자 수는?"
↓ 스키마 분석
- 나이 조건: Age_at_Histological_Diagnosis >= 50
- 흡연 조건: Smoking_status (흡연 상태)
- 돌연변이 조건: EGFR_mutation_status
↓ SQL 생성
SELECT COUNT(*) 
FROM clinical_genomic 
WHERE Age_at_Histological_Diagnosis >= 50 
  AND Smoking_status LIKE '%smoker%' 
  AND EGFR_mutation_status LIKE '%Mutant%'
```

### Text2SQL의 핵심 요소

1. **스키마 이해**: 테이블 구조와 컬럼명 파악
2. **의미 매핑**: 자연어 개념을 데이터베이스 필드로 연결
3. **조건 변환**: 자연어 조건을 SQL WHERE 절로 변환
4. **집계 함수**: 평균, 개수, 최대값 등의 통계 연산
5. **결과 해석**: SQL 결과를 자연어로 설명

이제 실제 DB Agent를 구현하여 이 과정을 확인해보겠습니다.


### 환경 설정

필요한 라이브러리와 의존성을 설치합니다.

In [ ]:
%pip install strands-agents strands-agents-tools psycopg2-binary --quiet

### PostgreSQL 데이터베이스와 연동하는 도구를 생성합니다.

다음 rds endpoint를 워크샵 메인 페이지 Outputs AuroraClusterEndpoint 값을 확인해 DB_CONFIG 변수에 할당 합니다.
<img src="https://raw.githubusercontent.com/jikang-jeong/aws-q-mem-mcp-server/refs/heads/main/rdb_sample0.png" width="800" alt="rds endpoint">



In [ ]:
# Strands Agents
from strands import Agent, tool
import psycopg2
import logging
import sys
from strands.models import BedrockModel
from collections import defaultdict

DB_CONFIG = {
    "host": "⚠️ 중요: RDS 엔드포인트 설정 Outputs Key: AuroraClusterEndpoint 값을 입력하세요.",
    "port": 5432,
    "database": "agentdb",
    "user": "dbadmin",
    "password": "postgres",
    "connect_timeout": 10
}

에이전트는 두 가지 주요 도구를 사용 합니다:
1. fetch_table_schema(): 데이터베이스 스키마 정보 조회
2. execute_postgres_query(): SQL 쿼리 실행


In [ ]:
logging.basicConfig(
    level=logging.INFO,
    format='%(filename)s:%(lineno)d | %(message)s',
    handlers=[logging.StreamHandler(sys.stderr)]
)
logger = logging.getLogger("internal_db")

@tool
def fetch_table_schema() -> str:

    """Postgres public 스키마의 테이블과 컬럼 정보를 조회하는 도구입니다.
    - 쿼리를 자동 생성할 때 테이블 구조가 필요하면 이 도구를 사용하세요.
    - 단순히 데이터 조회만 하는 경우에는 호출할 필요가 없습니다.
    """
    try:
        conn = psycopg2.connect(**DB_CONFIG)
        with conn.cursor() as cursor:
            cursor.execute("""
                           SELECT c.relname AS table_name, a.attname AS column_name,
                                  pg_catalog.format_type(a.atttypid, a.atttypmod) AS data_type
                           FROM pg_catalog.pg_attribute a
                                    JOIN pg_catalog.pg_class c ON a.attrelid = c.oid
                                    JOIN pg_catalog.pg_namespace n ON c.relnamespace = n.oid
                           WHERE a.attnum > 0 AND NOT a.attisdropped
                             AND n.nspname = 'public' AND c.relkind = 'r'
                           ORDER BY c.relname, a.attnum;
                           """)

            rows = cursor.fetchall()
            tables = defaultdict(list)
            for table_name, column_name, data_type in rows:
                tables[table_name].append(f"{column_name} ({data_type})")

            result = "\n".join([f"{tbl}: {', '.join(cols)}" for tbl, cols in tables.items()])
            conn.close()
            return result
    except Exception as e:
        return f"Error: {str(e)}"

@tool
def execute_postgres_query(query: str) -> str:

    """사용자가 요청한 SQL 쿼리를 실행하는 도구입니다.
    - 쿼리가 명확히 주어진 경우 이 도구만 호출하면 됩니다.
    - 쿼리를 자동 생성해야 하는 경우, 필요하다면 먼저 fetch_table_schema를 호출해 스키마를 확인한 뒤 실행하세요.

    Args:
        query: 실행할 SQL 쿼리문
    """
    try:
        conn = psycopg2.connect(**DB_CONFIG)
        cur = conn.cursor()
        cur.execute(query)
        result = cur.fetchall()
        conn.commit()
        return {
            "message": "\n".join(str(row) for row in result),
            "status": "success"
        }
    except Exception as e:
        return {
            "message": str(e)
        }
    finally:
        if 'cur' in locals():
            cur.close()
        if 'conn' in locals():
            conn.close()

In [ ]:
# Bedrock 모델 설정
model = BedrockModel(
    model_id="global.anthropic.claude-sonnet-4-5-20250929-v1:0",
    region_name="us-east-1"
)

### 데이터베이스 에이전트 생성
PostgreSQL 데이터베이스와 연동하는 Strands 에이전트를 생성합니다.
이 에이전트는 스키마조회와, 쿼리실행 두가지 도구를 사용하여 자연어 질문을 SQL 쿼리로 변환하고 실행합니다.

In [ ]:

# 사내 데이터베이스 탐색 전문 에이전트
dbAgent = Agent(
    tools=[fetch_table_schema, execute_postgres_query],
    name="PostgreSQL Assistant",
    model=model,
    system_prompt="""당신은 PostgreSQL 데이터베이스 전문가입니다.
사용자의 질문을 이해하고 적절한 SQL 쿼리를 생성하여 실행합니다. 생성쿼리, 실행계획, 방법, 이유는 각 도구실행전 설명후 실행하세요.
항상 도움이 되고 정확한 정보를 제공하며, 의료 데이터의 중요성을 인식하고 신중하게 분석하세요
결과를 명확하고 이해하기 쉽게 설명해주세요."""
)



## 데이터베이스 에이전트 테스트
데이터베이스 에이전트를 다양한 쿼리로 테스트해봅니다.


#### Test 1: 데이터베이스 연결 및 스키마 조회

In [ ]:
# internal db (postgres) 에이전트 테스트
print(" [Test 1] 데이터베이스 연결 및 테이블 확인")
db_result = dbAgent("데이터베이스에 어떤 테이블들이 있는지 알려주세요")
print("\n" + "="*50 + "\n")

#### Test 2: 간단한 데이터베이스 쿼리

In [ ]:
# internal db (postgres) 에이전트 테스트
print("⏰ [Test 2] 간단한 데이터베이스 쿼리 실행")
db_result = dbAgent("현재 데이터베이스 시간을 알려주세요")
print("\n" + "="*50 + "\n")

#### Test 3: 임상 데이터 분석

In [ ]:
# internal db (postgres) 에이전트 테스트
print(" [Test 3] 임상 데이터 분석 - 선암 환자 평균 연령")
db_result = dbAgent("선암(Adenocarcinoma) 진단을 받은 환자의 평균 연령은 얼마입니까?")
print("\n" + "="*50 + "\n")

#### Test 4: 복합 분석 쿼리

In [ ]:

# internal db (postgres) 에이전트 테스트
print(" [Test 4] 복합 조건 분석 - 50세 이상 환자 흡연 현황")
db_result = dbAgent("50세 이상 진단을 받은 환자의 수는 몇 명이며, 흡연 여부는 어떻게 됩니까?")
print("\n" + "="*50 + "\n")

#### Test 5: 생존 분석

In [ ]:

# internal db (postgres) 에이전트 테스트
print(" [Test 5] 생존 분석 - 화학요법 후 LRIG1 유전자 발현량 비교")
db_result = dbAgent("화학요법 후 생존한 환자와 사망한 환자의 LRIG1 유전자 발현량 평균을 비교해주세요")
print("\n" + "="*50 + "\n")

#### Test 6: 돌연변이 분석

In [ ]:

# internal db (postgres) 에이전트 테스트
print(" [Test 6] 돌연변이 분석 - EGFR 돌연변이 환자 생존율")
db_result = dbAgent("EGFR 돌연변이를 가진 환자들의 비율과 이들의 생존율을 분석해주세요")
print("\n" + "="*50 + "\n")


## 결론

이 노트북에서는 Strands Agents 프레임워크를 사용하여 PostgreSQL 데이터베이스와 연동하는 에이전트를 성공적으로 구현했습니다.

### 주요 구현 내용:

#### 1. Text2SQL 핵심 메커니즘
- **자연어 이해**: 사용자 질문의 의도 파악
- **스키마 매핑**: 자연어 개념을 데이터베이스 필드로 연결
- **SQL 생성**: 조건과 집계 함수를 포함한 정확한 쿼리 생성
- **결과 해석**: SQL 결과를 자연어로 설명

#### 2. 데이터베이스 도구 구현
- **fetch_table_schema**: 데이터베이스 스키마 탐색 도구
- **execute_postgres_query**: SQL 쿼리 실행 도구

#### 3. 에이전트 기능
- 자연어 질문을 SQL 쿼리로 변환 (Text2SQL)
- 임상 및 유전체 데이터 분석
- 생존 분석 및 돌연변이 분석
- 통계적 분석 및 패턴 발견

#### 4. 데이터셋 특징
- **chemotherapy_survival**: 화학요법 후 환자 생존 데이터
- **clinical_genomic**: 초기 단계 비소세포폐암 환자 코호트 데이터
- 종양 조직의 유전자 발현 데이터
- 환자의 임상 및 인구통계학적 정보

### Text2SQL의 실제 활용:

**변환 예시들:**
```
"선암 환자의 평균 연령" → SELECT AVG(Age_at_Histological_Diagnosis) FROM clinical_genomic WHERE Histology LIKE '%Adenocarcinoma%'
"EGFR 돌연변이 환자 수" → SELECT COUNT(*) FROM clinical_genomic WHERE EGFR_mutation_status LIKE '%Mutant%'
"생존한 환자의 LRIG1 평균" → SELECT AVG(LRIG1) FROM chemotherapy_survival WHERE Survival_Status = true
```


### 사용 예시:
- 선암종 환자들의 평균 연령 조회
- 편평세포암종 환자들의 가장 흔한 병리학적 T 병기 분석
- EGFR 돌연변이를 가진 환자들의 비율 계산
- 화학요법 후 생존율과 유전자 발현량 상관관계 분석

이 구현을 통해 의료 연구자들은 복잡한 SQL을 작성하지 않고도 자연어로 데이터베이스를 쿼리하고 분석할 수 있게 되었습니다. **Text2SQL 기술**이 DB Agent의 핵심 엔진 역할을 하여, 전문 지식이 없는 사용자도 데이터베이스와 자연스럽게 상호작용할 수 있습니다.